In [1]:
import pandas as pd
from datetime import datetime
import gspread as gs


In [12]:
# ruta del archvio xlsx a leer y escribir
file_path = "Llamadas.xlsx"
file_path_destino = "llamadas_procesado.xlsx"
credentials_filename ='\credenciales\client_secret_290615798724-sg1nk4latali6a8akhe169r66gg7hmd8.apps.googleusercontent.com.json'

gc = gs.oauth(credentials_filename)

# Fecha de inicio y fin de la revisión

fecha_inicial = datetime(2024, 8, 1)
fecha_final = datetime(2024, 8, 10)

# Lee el archivo Excel
df = pd.read_excel(file_path)

In [13]:
dias = {
    0: "Lunes",
    1: "Martes",
    2: "Miércoles",
    3: "Jueves",
    4: "Viernes",
    5: "Sábado",
    6: "Domingo"
}

# agregamos una columna con el día de la semana de la llamada

df["DiaSemana"] = df["Date"].dt.dayofweek.map(dias)


In [14]:
# clasificacmos las llamadas en madrugada, mañana, tarde y noche
def horario_llamadas(hora):
    if hora in range(8 ,13):
        return "Mañana"
    elif hora in range(13, 18):
        return "Tarde"
    elif hora in range(18, 23):
        return "Noche"
    else:
        return "Madrugada"
    
df["Horario"] = df["Date"].dt.hour.map(horario_llamadas)


In [15]:
# clasificamos de acuerdo a duration  
def duracion_llamadas(duracion):
    if duracion <= 5:
        return "Corta"
    elif duracion < 15:
        return "Media"
    elif duracion < 30:
        return "Larga"
    elif duracion >= 30:
        return "Muy Larga"
    else:
        return "Indefinida"
    
df["Franja_duracion"] = df["Duracion"].map(duracion_llamadas)

In [16]:
# analizamos la columna druacion

df["Franja_duracion"].value_counts()


Franja_duracion
Corta         1788
Media         1659
Larga          501
Muy Larga      163
Indefinida      68
Name: count, dtype: int64

In [7]:
#df.to_excel(file_path_destino, index=False)

In [23]:
sh = gc.create(file_path_destino)


In [25]:
# reemplazamos datetime por string
df = df.applymap(
    lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
    if isinstance(x, (pd.Timestamp, datetime)) and not pd.isnull(x) else x
)

# reemplazamos nulls por ''
df = df.applymap(lambda x: '' if pd.isnull(x) else x)

In [26]:
sh = gc.open(file_path_destino)
worksheet = sh.sheet1
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '125_kQTGguLjXUYeqQ-0c84DPdardr-e4Ig2wp-MFHsM',
 'updatedRange': "'Hoja 1'!A1:J4180",
 'updatedRows': 4180,
 'updatedColumns': 10,
 'updatedCells': 41800}